Все импорты должны выполняться, особенно если вы запускаете это внутри докера

In [1]:
%pylab inline
%config InlineBackend.figure_format = 'retina'
style.use('dark_background')
from IPython.display import clear_output
from copy import copy, deepcopy
from tqdm import tqdm_notebook as tm
from tqdm import trange
from glob import glob
import os
import sys
import warnings
warnings.filterwarnings("ignore")
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

Populating the interactive namespace from numpy and matplotlib


In [3]:
import pandas as pd
import pandas.io.sql as psql

Вспомогательные функции для запроса данных из базы

In [4]:
def get_conn():
    import psycopg2 as pg
    return pg.connect("dbname=vk_data user=vk_parser password=WegEWWXfedLf2YbS host=13.84.188.132")

def run_sql(query):
    import pandas.io.sql as psql
    res = psql.read_sql(query, get_conn())
    try:
        return res.set_index('uid')
    except:
        import warnings
        warnings.warn("[run_sql] UID wasn't selected, default index used")
        return res

def get_uid():
    return run_sql("SELECT uid FROM users WHERE data_loaded")

In [5]:
df = run_sql("SELECT uid FROM users WHERE first_name = 'Аня'")
df.head()

""
uid
21754523
67223827
190775618
6608545
2124713


In [6]:
df = run_sql("SELECT from_id, text FROM posts  WHERE (text <> '') AND (text LIKE '% да ладно тебе%')")
df.head()

,from_id,text
0,82380,"Ой, да ладно тебе;)) Стиснительный тут нашелся..."
1,36596923,ой да ладно тебе=)\nя думаю что ти думаешь что...
2,15759198,хд да ладно тебе)))))))я же шучу насчет кулисо...
3,1480663,"ой, да ладно тебе))))))) кто бы говорил!!!!"
4,1088981,"ахахах!! да ладно тебе издеваться, Ир!))))))))..."


### Извлекаем эмодзи из сообщений

In [7]:
# !pip3 install emoji
import emoji

In [8]:
def getEmojiFromString(text):
    listText = list(text)
    emojiList = [c for c in listText if c in emoji.UNICODE_EMOJI]
    return np.unique(emojiList, return_counts=True)

def getSarcasmEmojiFromString(text):
    sarcasmEmoji = ['🙃', '🐸', '💁', '🌝']
    emojiList = [c for c in text if c in emoji.UNICODE_EMOJI]
    sarcasmEmojiList = [c for c in emojiList if c in sarcasmEmoji]
    return np.unique(sarcasmEmojiList, return_counts=True)

def getNegativeEmojiFromString(text):
    negativeEmoji = ['😠', '😡', '😤', '👿', '👎', '👊', '👺', '🤬', '😈', '💀', '☠️', '🔫', '😕', '🙅', '😾', '🖕']
    emojiList = [c for c in text if c in emoji.UNICODE_EMOJI]
    sarcasmEmojiList = [c for c in emojiList if c in negativeEmoji]
    return np.unique(sarcasmEmojiList, return_counts=True)

def getNegativeEmojiValueFromString(s):
    negEmoji = getNegativeEmojiFromString(s)
    return negEmoji[1].sum()

### Выделяем "сарказм" из текста

In [9]:
import string 
def removePunctuationFromString(s):
    return s.translate(str.maketrans('', '', string.punctuation))

def getSarcasmPhraseFromString(s):
    sarcasmVocabulary = ['ну да конечно', 'да что ты говоришь', 'да ладно', 'спасибо кэп', 'что бы мы без тебя делали',
                         'ага конечно', 'прям сразу', 'прямо сразу', 'ой конечно', 'ой ладно', 'да что ты', 'ой да конечно']
    strClean = removePunctuationFromString(s)
    sarcasmList = []
    for sarcasmExpression in sarcasmVocabulary:
        if sarcasmExpression in s:
            sarcasmList.append(sarcasmExpression)
    return sarcasmList

def getSarcasmValueFromString(s):
    sarcasmPhrase = getSarcasmPhraseFromString(s)
    sarcasmEmoji = getSarcasmEmojiFromString(s)
    return int(len(sarcasmPhrase) or len(sarcasmEmoji[0]))

In [16]:
s = 'ой да ладно прям стесняшка!))))'
print(removePunctuationFromString(s))
print(getSarcasmPhraseFromString(s))
print(getSarcasmValueFromString(s))


ой да ладно прям стесняшка
['да ладно']
1


In [14]:
def getPostsByUid(uid):
    sqlQuery = "SELECT from_id, text FROM posts  WHERE (text <> '') AND (from_id = " + str(uid) + ")"
    return run_sql(sqlQuery).text

In [15]:
getPostsByUid(15759198)

0                                       хм...в смысле?)
1     не я все так удаляю))оставляю ток картинки и м...
2                    аххах ну это все для дебя детка)))
3     ой не повериш как я искала искала хд))а потом ...
4                           аххаха вот сейчас)))))хд)))
5                            пашка!!!а я тебя нашла))хд
6                                  урааа пасипки зай)))
7     хд да ладно тебе)))))))я же шучу насчет кулисо...
8     хд ну пашечка ну солнышко..ну малыш..ну кися))...
9                 аххах а ты меня возешь с собой???))))
10    пасип зайк))обязательно буду)))хахах)))*лыбить...
11    ооххохо прикольно))я та тоже не плохо это а ко...
12    оооо пашка ну ты отрастил патлы))))мужичек))))...
Name: text, dtype: object